Set FLAGS

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

In [ ]:
class FLAGS(object):
    """ """
    """
    General settings
    """
    input_size = (1024, 1024)
    heatmap_size = 128
    cpm_stages = 4
    joint_gaussian_variance = 1.0
    center_radius = 21
    num_of_joints = 8
    color_channel = 'RGB'
    normalize = True
    use_gpu = True
    gpu_id = 0
    
    gradient_clipping = True # gradient clipping

    """
    Demo settings
    """
    # 'MULTI': show multiple stage heatmaps
    # 'SINGLE': show last stage heatmap
    # 'Joint_HM': show last stage heatmap for each joint
    # 'image or video path': show detection on single image or video
    DEMO_TYPE = 'SINGLE'

    model_path = 'cpm_hand'
    cam_id = 0

    use_kalman = True
    kalman_noise = 0.03
    keypoints_order = ["TAIL_NOTCH",
                        "ADIPOSE_FIN",
                        "UPPER_LIP",
                        "ANAL_FIN",
                        "PELVIC_FIN",
                        "EYE",
                        "PECTORAL_FIN",
                        "DORSAL_FIN"]

    """
    Training settings
    """
    network_def = 'fish_test'
    train_img_dir = ''
    val_img_dir = ''
    bg_img_dir = ''
    pretrained_model = 'fish_test'
    batch_size = 4
    init_lr = 0.001
    lr_decay_rate = 0.45
    lr_decay_step = 8000
    augmentation = None
    buffer_range = [int(n) for n in np.arange(100, 600, 100)] # useless if crop = False
    crop = False # crop input image based on keypoints - for GTSF only
#     augmentation = albu.Compose([albu.HorizontalFlip(p=0.5),
# #                                  albu.Rotate(limit=10, p=1.0)
#                                 ], 
#                                  p=1.0,
#                                  keypoint_params={'format': 'xy'})
    
    epochs=200

    hnm = True  # Make sure generate hnm files first
    do_cropping = True

    """
    For Freeze graphs
    """
    output_node_names = 'stage_3/mid_conv7/BiasAdd:0'

## Load data

In [ ]:
from utils.utils import DataGenerator, data_generator
import time
import json

In [ ]:
annotations = json.load(open('/root/data/bati/labels/labels_2019-05-10.json'))

In [ ]:
for ann in annotations:
    ann['local_path'] = ann['local_path'].replace('/app', '/root')

## New functions

In [ ]:
from utils.cpm_utils import make_gaussian
from utils.utils import load_image_keypoints

In [ ]:
def make_heatmap_single(input_size, heatmap_size, gaussian_variance, batch_joints):
    scale_factor = input_size // heatmap_size
    gt_heatmap_np = []
    invert_heatmap_np = np.ones(shape=(heatmap_size, heatmap_size))
    for j in range(batch_joints.shape[0]):
        cur_joint_heatmap = make_gaussian(heatmap_size,
                                          gaussian_variance,
                                          center=(batch_joints[j] // scale_factor))
        gt_heatmap_np.append(cur_joint_heatmap)
        invert_heatmap_np -= cur_joint_heatmap
    gt_heatmap_np.append(invert_heatmap_np)
    gt_heatmap_np = np.array(gt_heatmap_np)
    gt_heatmap_np = np.transpose(gt_heatmap_np, (1, 2, 0))
    return gt_heatmap_np

In [ ]:
def load_image_keypoints_gaussian(annotation):
#     print('test')
    image, keypoints = load_image_keypoints(annotation, FLAGS)
    heatmap = make_heatmap_single(FLAGS.input_size[0], 
                                FLAGS.heatmap_size, 
                                FLAGS.joint_gaussian_variance,
                                keypoints)
    return image, keypoints, heatmap

In [ ]:
# start = time.time()

# end = time.time()
# print(end-start)

# New generator

In [ ]:
import keras
from multiprocessing import Pool
import random
from utils.utils import DataGeneratorAsync

In [ ]:
random.shuffle(annotations)

In [ ]:
# class DataGenerator(keras.utils.Sequence):
#     'Generates data for Keras'
#     def __init__(self, annotations_set, FLAGS):
#         'Initialization'
#         start = time.time()
#         self.annotations = annotations_set
#         self.on_epoch_end()
        
#     def __len__(self):
#         'Denotes the number of batches per epoch'
#         return len(self.annotations) // self.FLAGS.batch_size

#     def __getitem__(self, index):
#         'Generate one batch of data'
#         pool = Pool(4)
#         data = pool.map(load_image_keypoints_gaussian, 
#                         self.annotations[index*FLAGS.batch_size: (index+1)*FLAGS.batch_size]
#                        )
#         pool.close()
#         pool.join()
#         images = np.array([d[0] for d in data])
#         keypoints = np.array([d[1] for d in data])
#         heatmaps = np.array([d[2] for d in data])
#         return images, keypoints, heatmaps

#     def on_epoch_end(self):
#         'Updates indexes after each epoch'
#         np.random.shuffle(self.annotations)

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, annotations_set, FLAGS):
        'Initialization'
        start = time.time()
        self.annotations = annotations_set
        self.on_epoch_end()
        self.FLAGS = FLAGS
        pool = Pool(8)
        self.data = pool.map_async(load_image_keypoints_gaussian, self.annotations[:100])
        pool.close()
        pool.join()
        
        self.batches = []
        self._create_batches()
        end = time.time()
        print('Data is ready for training {}'.format(end-start))
        
    def _create_batches(self):
        data = self.data.get()
        images = [t[0] for t in data]
        keypoints = [t[1] for t in data]
        heatmaps = [t[2] for t in data]
        
        batch_images = [images[k:k+self.FLAGS.batch_size] for k in range(0, len(images), self.FLAGS.batch_size)]
        batch_keypoints = [keypoints[k:k+self.FLAGS.batch_size] for k in range(0, len(keypoints), self.FLAGS.batch_size)]
        batch_heatmaps = [heatmaps[k:k+self.FLAGS.batch_size] for k in range(0, len(heatmaps), self.FLAGS.batch_size)]
        
        for (img, kp, hm) in zip(batch_images, batch_keypoints, batch_heatmaps):
            self.batches.append([np.array(img), 
                                 np.array(batch_keypoints), 
                                 np.array(batch_heatmaps)])
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return len(self.annotations) // self.FLAGS.batch_size

    def __getitem__(self, index):
        'Generate one batch of data'
        return self.batches[index]

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        np.random.shuffle(self.annotations)

In [ ]:
random.shuffle(annotations)
gen = DataGenerator(annotations, FLAGS)

In [ ]:
test = gen[0]

In [ ]:
start = time.time()
for i in range(25):
    print(i)
    start = time.time()
    output = gen[i]
    
    print(output[0].shape)
    end = time.time()
    # time.sleep(0.1)
#     print("Loading time: {}".format((end - start)))
end = time.time()
print("Loading time: {}".format((end - start)))